### Problem 1.

numpy logistic regression

https://yamalab.tistory.com/79

https://pythonkim.tistory.com/18


### Problem 2.

실제 데이터 > 다중선형회귀

In [ ]:
import numpy as np

# 변수(x, y) 설정
x_train = np.array([]) # array comprehension으로 작성하기
y_train = np.array([]) # array comprehension으로 작성하기

# parameter(가중치, 편향) 초기화
W = 0.0
b = 0.0

# 최소제곱법 작성하기 위한 변수 저장(데이터 개수, epochs, learning_rate)
n_data = len(x_train)
epochs = 5000
learning_rate = 0.01

# 최소제곱법 실행
for i in range(epochs):
    # 가설함수 세우기
    hypothesis = x_train * W + b
    # 비용함수 세우기
    cost = np.sum((hypothesis - y_train) ** 2) / n_data
    # 경사하강법 구현
    gradient_w = np.sum((W * x_train - y_train + b) * 2 * x_train) / n_data
    gradient_b = np.sum((W * x_train - y_train + b) * 2) / n_data
    W -= learning_rate * gradient_w
    b -= learning_rate * gradient_b
    
    # 결과 출력
    if i % 100 == 0:
        print('Epoch ({:10d}/{:10d}) cost: {:10f}, W: {:10f}, b:{:10f}'.format(i, epochs, cost, W, b))

# 최종 결과 출력
print('W: {:10f}'.format(W))
print('b: {:10f}'.format(b))
print('result : ')
print(x_train * W + b)